In [37]:
# Working over image for number of kills 
import numpy as np
import matplotlib.pyplot as plt
import pytesseract
import imageio
import os
import copy
from PIL import Image

from scipy.ndimage import gaussian_filter
from skimage import data
from skimage import img_as_float
from skimage.morphology import reconstruction


# Convert to float: Important for subtraction later which won't work with uint8


def extract_text(image, ratio=[0.697,0.744,0.468,0.534]):
    #ratio=[0.697,0.746,0.468,0.534]
    image = np.asarray(image)
    y1 = int(image.shape[0] * ratio[0])
    y2 = int(image.shape[0] * ratio[1])
    x1 = int(image.shape[1] * ratio[2])
    x2 = int(image.shape[1] * ratio[3])
    image = image[y1:y2, x1:x2]
    image = Image.fromarray(np.uint8(image))

    basewidth = 1000
    wpercent = (basewidth / float(image.size[0]))
    hsize = int((float(image.size[1]) * float(wpercent)))
    image_resized = image.resize((basewidth, hsize), Image.ANTIALIAS)
    image = gaussian_filter(image, 0.4)

    seed = np.copy(image)
    seed[1:-1, 1:-1] = image.min()
    mask = image

    dilated = reconstruction(seed, mask, method='dilation')

    img_mid = image - dilated

    img_sub_dilated = img_mid
    image_1 = Image.fromarray(np.uint8(img_sub_dilated))
    image_test = Image.fromarray(np.uint8(image))
    text = pytesseract.image_to_string(image_1)
    
    return text.encode('utf-8')

In [38]:
import os
import cv2
import copy
from PIL import Image
from scipy.ndimage import gaussian_filter
from skimage import data
from skimage import img_as_float
from skimage.morphology import reconstruction


# Convert to float: Important for subtraction later which won't work with uint8
def extract_text_otsu(image, ratio=[0.697,0.744,0.468,0.534]):
    #ratio=[0.697,0.746,0.468,0.534]
    image = np.asarray(image)
    image.astype('uint8')
    y1 = int(image.shape[0] * ratio[0])
    y2 = int(image.shape[0] * ratio[1])
    x1 = int(image.shape[1] * ratio[2])
    x2 = int(image.shape[1] * ratio[3])
    image = image[y1:y2, x1:x2]
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image,(70,70), interpolation = cv2.INTER_CUBIC)
    blur = cv2.GaussianBlur(image,(5,5),0)
    ret,th = cv2.threshold(copy.copy(np.uint8(blur)),0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    image = Image.fromarray(np.uint8(th))
    text = pytesseract.image_to_string(image)
    
    return text.encode('utf-8')
   

In [39]:
# Working extraction from video
import imageio
import math
import numpy as np
import re
from matplotlib import pyplot as plt
imageio.plugins.ffmpeg.download()

def find_kills_images_from_stream(stream_url, delay):
            """
            Sample images from a live stream at regular intervals.
            :param stream_url: url where the stream is located
            :param delay: millisecond delay between sampled images
            :return: image_batch - NxCxHxW - (num images)x(dims of image) converted w/ image_to_caffe_format
            """
            # Save images into temporary directory specific to this stream.
            print "Reading from video stream as numpy array..."
            video_reader = imageio.get_reader(stream_url)
            video_info = video_reader.get_meta_data()
            duration = video_info['duration']
            delta = float(video_info['duration']) / video_info['nframes']
            frame_times = []
            seek_time = float(0)

            while seek_time < video_info['duration']:
                frame_times.append(seek_time)
                seek_time+=(float(delay)/1000)
            frame_indices = [int(math.floor(float(frame_time) / delta)) for frame_time in frame_times]
            print "Extracting frames by reading through using video_reader"
            ocr_response_frames = []
            kill_times = []
            kill_occured = False
            number_of_kills = set()
            try:
                for ind, frame in enumerate(video_reader):
                    sampled_frame = []                    
                    if ind in frame_indices:
                        ocr_response = dict()
                        frame_time = frame_times[frame_indices.index(ind)]
                        frame = np.asarray(frame)
                        new_frame = copy.copy(frame)
                        # Extract number of kills
                        ratio = [0.697,0.744,0.468,0.534]
                        ocr_text = extract_text(new_frame,ratio=ratio)

                        if ('kill' in ocr_text) and any(str.isdigit(c) for c in ocr_text):
                            kill = ''
                            prev_ind = None
                            for curr_ind, c in enumerate(ocr_text):                                    
                                if str.isdigit(c) and (prev_ind==None or prev_ind+1==curr_ind):
                                    kill+=c
                                    prev_ind=curr_ind
                            if kill not in number_of_kills:    
                                number_of_kills.add(kill)
                                kill_times.append(frame_time)
                                kill_occured = True
            except Exception as e:
                print e
                print "There was an error reading a frame. Other frames available in sampled_frames."

            return kill_occured, kill_times, duration

In [40]:
# Working extraction from video
import imageio
import math
import numpy as np
import re
from matplotlib import pyplot as plt
imageio.plugins.ffmpeg.download()

def index_images_from_stream(stream_url, delay):
            """
            Sample images from a live stream at regular intervals.
            :param stream_url: url where the stream is located
            :param delay: millisecond delay between sampled images
            :return: image_batch - NxCxHxW - (num images)x(dims of image) converted w/ image_to_caffe_format
            """
            # Save images into temporary directory specific to this stream.
            print "Reading from video stream and as numpy array..."
            video_reader = imageio.get_reader(stream_url)
            video_info = video_reader.get_meta_data()
            duration = video_info['duration']
            delta = float(video_info['duration']) / video_info['nframes']
            frame_times = []
            seek_time = float(0)

            while seek_time < video_info['duration']:
                frame_times.append(seek_time)
                seek_time+=(float(delay)/1000)
            frame_indices = [int(math.floor(float(frame_time) / delta)) for frame_time in frame_times]
            print "Extracting frames by reading through using video_reader"
            ocr_response_frames = []
            ocr_frame_times = []
            number_of_kills = set()
            try:
                for ind, frame in enumerate(video_reader):
                    sampled_frame = []                    
                    if ind in frame_indices:
                        ocr_response = dict()
                        frame_time = frame_times[frame_indices.index(ind)]
                        frame = np.asarray(frame)
                        new_frame = copy.copy(frame)
                        # Extract number of kills
                        ratio = [0.697,0.744,0.468,0.534]
                        ocr_text = extract_text(new_frame,ratio=ratio)

                        if ('kill' in ocr_text) and any(str.isdigit(c) for c in ocr_text):
                                kill = ''
                                prev_ind = None
                                for curr_ind, c in enumerate(ocr_text):                                    
                                    if str.isdigit(c) and (prev_ind==None or prev_ind+1==curr_ind):
                                        kill+=c
                                        prev_ind=curr_ind
                                if kill not in number_of_kills:    
                                    number_of_kills.add(kill)
                                    # Extract line        
                                    ratio = [0.6638,0.6986,0.3539,0.6484]
                                    ocr_text = extract_text(new_frame,ratio=ratio)
                                    alive = ''
                                    prev_ind = 0                           
                                    ocr_words = ocr_text.split(' ')
                                    person_killed = ocr_words[2]
                                    weapon = re.sub("[^a-zA-Z]+", "", ocr_words[-3])
                                    alive = ocr_words[-2]
                                    ocr_response["frame_time"] = frame_time
                                    ocr_response["kill"] = kill
                                    ocr_response["alive"] = alive
                                    ocr_response["person_killed"] = person_killed
                                    ocr_response["weapon_used"] = weapon
                                    ocr_response_frames.append(ocr_response)
            except Exception as e:
                print e
                print "There was an error reading a frame. Other frames available in sampled_frames."

            return ocr_response_frames, duration

In [41]:
import time
stream_url = 'http://d1playscdntv-a.akamaihd.net/video/GvaagNrIs0F/processed/720H.mp4'
start = time.time()
kill_occured, time_of_kills, duration = find_kills_images_from_stream(stream_url, 1000)

print "Did any kill occur?", kill_occured
print "When all did kill occur?", time_of_kills
m, s = divmod(duration, 60)
h, m = divmod(m, 60)
print "duration of video", "%d:%02d:%02d" % (h, m, s)
m, s = divmod(time.time() - start, 60)
h, m = divmod(m, 60)
print "time for processing", "%d:%02d:%02d" % (h, m, s) 

Reading from video stream as numpy array...
Extracting frames by reading through using video_reader
Did any kill occur? True
When all did kill occur? [24.0]
duration of video 0:00:27
time for processing 0:00:13


In [42]:
import time
stream_url = 'http://d0playscdntv-a.akamaihd.net/video/Gu_vRIV7mJg/processed/1080H.mp4'
start = time.time()
kill_occured, time_of_kills, duration = find_kills_images_from_stream(stream_url, 1000)

print "Did any kill occur?", kill_occured
print "When all did kill occur?", time_of_kills
m, s = divmod(duration, 60)
h, m = divmod(m, 60)
print "duration of video", "%d:%02d:%02d" % (h, m, s)
m, s = divmod(time.time() - start, 60)
h, m = divmod(m, 60)
print "time for processing", "%d:%02d:%02d" % (h, m, s) 

Reading from video stream as numpy array...
Extracting frames by reading through using video_reader
Did any kill occur? True
When all did kill occur? [39.0]
duration of video 0:02:00
time for processing 0:01:40


In [43]:
import time
stream_url = 'http://d1playscdntv-a.akamaihd.net/video/GvaagNrIs0F/processed/720H.mp4'
start = time.time()
response, duration = index_images_from_stream(stream_url, 1000)

print "response", response
m, s = divmod(duration, 60)
h, m = divmod(m, 60)
print "duration of video", "%d:%02d:%02d" % (h, m, s)
m, s = divmod(time.time() - start, 60)
h, m = divmod(m, 60)
print "time for processing", "%d:%02d:%02d" % (h, m, s) 

Reading from video stream and as numpy array...
Extracting frames by reading through using video_reader
response [{'person_killed': 'Kellumien', 'weapon_used': 'AKM', 'kill': '3', 'alive': '63', 'frame_time': 24.0}]
duration of video 0:00:27
time for processing 0:00:14


In [44]:
import time
stream_url = 'http://d0playscdntv-a.akamaihd.net/video/Gu_vRIV7mJg/processed/1080H.mp4'
start = time.time()
response, duration = index_images_from_stream(stream_url, 1000)

print "response", response
m, s = divmod(duration, 60)
h, m = divmod(m, 60)
print "duration of video", "%d:%02d:%02d" % (h, m, s)
m, s = divmod(time.time() - start, 60)
h, m = divmod(m, 60)
print "time for processing", "%d:%02d:%02d" % (h, m, s) 

Reading from video stream and as numpy array...
Extracting frames by reading through using video_reader
response [{'person_killed': 'Broodwich', 'weapon_used': 'withetmr', 'kill': '3', 'alive': '2', 'frame_time': 39.0}]
duration of video 0:02:00
time for processing 0:01:51


In [43]:
import time
stream_url = 'http://d1playscdntv-a.akamaihd.net/video/Gvcf6o1bWwE/processed/1080.mp4'
start = time.time()
response, duration = index_images_from_stream(stream_url, 1000)

print "response", response
m, s = divmod(duration, 60)
h, m = divmod(m, 60)
print "duration of video", "%d:%02d:%02d" % (h, m, s)
m, s = divmod(time.time() - start, 60)
h, m = divmod(m, 60)
print "time for processing", "%d:%02d:%02d" % (h, m, s) 

Reading from video stream and saving it as numpy array...
Calculate the times and frames to sample.
Extracting frames reading through using video_reader
response []
duration of video 0:02:28
time for processing 0:01:20


In [44]:
import time
stream_url = 'https://s3-us-west-1.amazonaws.com/acusense-images/test/2017_05_15_15_41_51-ses.mp4'
start = time.time()
response, duration = index_images_from_stream(stream_url, 1000)

print "response", response
m, s = divmod(duration, 60)
h, m = divmod(m, 60)
print "duration of video", "%d:%02d:%02d" % (h, m, s)
m, s = divmod(time.time() - start, 60)
h, m = divmod(m, 60)
print "time for processing", "%d:%02d:%02d" % (h, m, s)     

Reading from video stream and saving it as numpy array...
Calculate the times and frames to sample.
Extracting frames reading through using video_reader
Could not read frame 100405:
Frame is 0 bytes, but expected 6220800.
=== stderr ===
ffmpeg version N-86111-ga441aa90e8-static http://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 5.4.1 (Debian 5.4.1-8) 20170304
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc-5 --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gray --enable-libass --enable-libfreetype --enable-libfribidi --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-librtmp --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --e